# Banking Transaction Anomaly Detection System
Complete Pipeline: From EDA to Model Evaluation
This notebook performs comprehensive anomaly detection on banking transaction data.

Step 1: Setup and Installation


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Machine Learning Libraries
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.ensemble import IsolationForest, RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score, 
                           precision_recall_curve, average_precision_score, f1_score,
                           roc_curve, auc, precision_score, recall_score, accuracy_score)
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN, KMeans

import xgboost as xgb
import lightgbm as lgb
from scipy import stats
from scipy.stats import zscore
import joblib
import pickle

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Libraries imported successfully!")


✅ Libraries imported successfully!


# Step 2: Load Data from CSV

In [4]:
# Loading data from CSV files

try:
    # Load account data
    df_accounts = pd.read_csv('../data/raw/Account.csv', low_memory=False)
    print(f"✅ Accounts data loaded: {df_accounts.shape}")
    
except FileNotFoundError as e:
    print(f"❌ Error loading files: {e}")
    print("Creating sample data for demonstration...")


✅ Accounts data loaded: (10000, 9)


In [5]:
# Display basic information
print("="*80)
print("DATA OVERVIEW")
print("="*80)

print("\n📊 Accounts Data:")
print("-" * 40)
print(df_accounts.info())
print("\nFirst 5 rows:")
display(df_accounts.head())

DATA OVERVIEW

📊 Accounts Data:
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ACCOUNT_NUMBER     9987 non-null   float64
 1   CUSTOMER_CODE      10000 non-null  int64  
 2   EOM                10000 non-null  int64  
 3   CURRENTBALANCELCY  10000 non-null  int64  
 4   Monthly_Change     9208 non-null   float64
 5   Days_Since_Credit  10000 non-null  int64  
 6   Days_Since_Debit   10000 non-null  int64  
 7   Fraud_Flag         10000 non-null  int64  
 8   DORMANCYDAYS       10000 non-null  int64  
dtypes: float64(2), int64(7)
memory usage: 703.3 KB
None

First 5 rows:


,ACCOUNT_NUMBER,CUSTOMER_CODE,EOM,CURRENTBALANCELCY,Monthly_Change,Days_Since_Credit,Days_Since_Debit,Fraud_Flag,DORMANCYDAYS
0,NaN,478095340,202401,0,NaN,81844,81844,0,730
1,NaN,478095340,202402,0,0.00,81873,81873,0,730
2,NaN,478095340,202501,0,0.00,82210,82210,0,730
3,NaN,478095340,202502,0,0.00,82238,82238,0,730
4,NaN,478095340,202503,0,0.00,82269,82269,0,730


# Step 2.1: Data Quality Assessment and Cleaning

In [13]:
def assess_data_quality(df_accounts):
    """Comprehensive data quality assessment"""
    print("="*80)
    print("DATA QUALITY ASSESSMENT")
    print("="*80)
    
    # 1. Data Types Check
    print("\n📊 DATA TYPES:")
    print("-" * 40)
    print(df_accounts.dtypes)
    
    # 2. Missing Values Analysis
    print("\n🔍 MISSING VALUES:")
    print("-" * 40)
    missing_summary = pd.DataFrame({
        'Missing_Count': df_accounts.isnull().sum(),
        'Missing_%': (df_accounts.isnull().sum() / len(df_accounts)) * 100,
        'Unique_Values': df_accounts.nunique(),
        'Data_Type': df_accounts.dtypes
    })
    display(missing_summary)
    
    # 3. Zero Values Analysis
    print("\n🔍 ZERO VALUES ANALYSIS:")
    print("-" * 40)
    numeric_cols = df_accounts.select_dtypes(include=[np.number]).columns
    zero_counts = {}
    
    for col in numeric_cols:
        zero_pct = (df_accounts[col] == 0).sum() / len(df_accounts) * 100
        if zero_pct > 50:  # Highlight columns with >50% zeros
            zero_counts[col] = zero_pct
    
    if zero_counts:
        print("⚠️  Columns with >50% zero values:")
        for col, pct in zero_counts.items():
            print(f"   {col}: {pct:.1f}% zeros")
    else:
        print("✅ No columns with excessive zero values")
    
    # 4. Data Distribution Analysis
    print("\n📈 DATA DISTRIBUTION (First 10 rows):")
    print("-" * 40)
    display(df_accounts.head(10))
    
    # 5. Target Variable Analysis
    if 'Fraud_Flag' in df_accounts.columns:
        print("\n🎯 TARGET VARIABLE DISTRIBUTION:")
        print("-" * 40)
        fraud_dist = df_accounts['Fraud_Flag'].value_counts()
        print(f"Non-Fraud: {fraud_dist.get(0, 0):,} ({fraud_dist.get(0, 0)/len(df_accounts)*100:.2f}%)")
        print(f"Fraud: {fraud_dist.get(1, 0):,} ({fraud_dist.get(1, 0)/len(df_accounts)*100:.2f}%)")
    
    return df_accounts, missing_summary

# Assess data quality
df_clean, quality_report = assess_data_quality(df_accounts)

DATA QUALITY ASSESSMENT

📊 DATA TYPES:
----------------------------------------
ACCOUNT_NUMBER       float64
CUSTOMER_CODE          int64
EOM                    int64
CURRENTBALANCELCY      int64
Monthly_Change       float64
Days_Since_Credit      int64
Days_Since_Debit       int64
Fraud_Flag             int64
DORMANCYDAYS           int64
dtype: object

🔍 MISSING VALUES:
----------------------------------------


,Missing_Count,Missing_%,Unique_Values,Data_Type
ACCOUNT_NUMBER,13,0.13,791,float64
CUSTOMER_CODE,0,0.00,785,int64
EOM,0,0.00,12,int64
CURRENTBALANCELCY,0,0.00,30,int64
Monthly_Change,792,7.92,25,float64
Days_Since_Credit,0,0.00,1104,int64
Days_Since_Debit,0,0.00,996,int64
Fraud_Flag,0,0.00,1,int64
DORMANCYDAYS,0,0.00,1,int64



🔍 ZERO VALUES ANALYSIS:
----------------------------------------
⚠️  Columns with >50% zero values:
   CURRENTBALANCELCY: 99.3% zeros
   Monthly_Change: 91.8% zeros
   Fraud_Flag: 100.0% zeros

📈 DATA DISTRIBUTION (First 10 rows):
----------------------------------------


,ACCOUNT_NUMBER,CUSTOMER_CODE,EOM,CURRENTBALANCELCY,Monthly_Change,Days_Since_Credit,Days_Since_Debit,Fraud_Flag,DORMANCYDAYS
0,NaN,478095340,202401,0,NaN,81844,81844,0,730
1,NaN,478095340,202402,0,0.00,81873,81873,0,730
2,NaN,478095340,202501,0,0.00,82210,82210,0,730
3,NaN,478095340,202502,0,0.00,82238,82238,0,730
4,NaN,478095340,202503,0,0.00,82269,82269,0,730
5,NaN,478095340,202503,0,0.00,82269,82269,0,730
6,NaN,478095340,202504,0,0.00,82299,82299,0,730
7,NaN,478095340,202505,0,0.00,82330,82330,0,730
8,NaN,478095340,202506,0,0.00,82360,82360,0,730
9,NaN,478095340,202507,0,0.00,82391,82391,0,730



🎯 TARGET VARIABLE DISTRIBUTION:
----------------------------------------
Non-Fraud: 10,000 (100.00%)
Fraud: 0 (0.00%)


# Step 2.2: Clean and Transform Data

In [14]:
def clean_and_transform_data(df_accounts):
    """Clean and transform the data"""
    print("="*80)
    print("DATA CLEANING & TRANSFORMATION")
    print("="*80)
    
    df_clean = df_accounts.copy()
    
    # 1. Fix ACCOUNT_NUMBER (convert to string, handle NaN)
    print("\n1. Fixing ACCOUNT_NUMBER...")
    if 'ACCOUNT_NUMBER' in df_clean.columns:
        # Convert to string, handle NaN
        df_clean['ACCOUNT_NUMBER'] = df_clean['ACCOUNT_NUMBER'].fillna('UNKNOWN')
        df_clean['ACCOUNT_NUMBER'] = df_clean['ACCOUNT_NUMBER'].astype(str).str.replace('.0', '', regex=False)
        print(f"   Account numbers converted to string")
    
    # 2. Handle CURRENTBALANCELCY zeros
    print("\n2. Analyzing CURRENTBALANCELCY...")
    if 'CURRENTBALANCELCY' in df_clean.columns:
        zero_balance_pct = (df_clean['CURRENTBALANCELCY'] == 0).sum() / len(df_clean) * 100
        print(f"   Zero balance accounts: {zero_balance_pct:.1f}%")
        
        if zero_balance_pct > 90:
            print("   ⚠️  Warning: Most accounts have zero balance - consider filtering")
        else:
            print("   ✅ Balance distribution appears reasonable")
    
    # 3. Fix EOM format (convert to datetime)
    print("\n3. Converting EOM to datetime...")
    if 'EOM' in df_clean.columns:
        try:
            df_clean['EOM'] = pd.to_datetime(df_clean['EOM'].astype(str), format='%Y%m')
            print(f"   Date range: {df_clean['EOM'].min()} to {df_clean['EOM'].max()}")
        except:
            print("   ⚠️  Could not convert EOM to datetime")
    
    # 4. Handle extreme Days_Since values
    print("\n4. Checking Days_Since values...")
    date_cols = ['Days_Since_Credit', 'Days_Since_Debit']
    for col in date_cols:
        if col in df_clean.columns:
            max_days = df_clean[col].max()
            if max_days > 365 * 10:  # More than 10 years
                print(f"   ⚠️  {col} has extreme values (max: {max_days} days)")
                # Cap at reasonable value (e.g., 5 years)
                df_clean[col] = df_clean[col].clip(upper=365*5)
                print(f"   Values capped at 5 years (1825 days)")
    
    # 5. Handle missing Monthly_Change
    print("\n5. Handling missing Monthly_Change...")
    if 'Monthly_Change' in df_clean.columns:
        missing_pct = df_clean['Monthly_Change'].isnull().sum() / len(df_clean) * 100
        print(f"   Missing Monthly_Change: {missing_pct:.1f}%")
        
        if missing_pct > 10:
            # Fill with 0 for missing (assuming no change)
            df_clean['Monthly_Change'] = df_clean['Monthly_Change'].fillna(0)
            print(f"   Missing values filled with 0")
    
    print(f"\n✅ Cleaning complete. New shape: {df_clean.shape}")
    print(f"   First 5 rows after cleaning:")
    display(df_clean.head())
    
    return df_clean

# Clean the data
df_cleaned = clean_and_transform_data(df_accounts)

DATA CLEANING & TRANSFORMATION

1. Fixing ACCOUNT_NUMBER...
   Account numbers converted to string

2. Analyzing CURRENTBALANCELCY...
   Zero balance accounts: 99.3%
   ⚠️  Warning: Most accounts have zero balance - consider filtering

3. Converting EOM to datetime...
   Date range: 2024-01-01 00:00:00 to 2025-10-01 00:00:00

4. Checking Days_Since values...
   ⚠️  Days_Since_Credit has extreme values (max: 82483 days)
   Values capped at 5 years (1825 days)
   ⚠️  Days_Since_Debit has extreme values (max: 82483 days)
   Values capped at 5 years (1825 days)

5. Handling missing Monthly_Change...
   Missing Monthly_Change: 7.9%

✅ Cleaning complete. New shape: (10000, 9)
   First 5 rows after cleaning:


,ACCOUNT_NUMBER,CUSTOMER_CODE,EOM,CURRENTBALANCELCY,Monthly_Change,Days_Since_Credit,Days_Since_Debit,Fraud_Flag,DORMANCYDAYS
0,UNKNOWN,478095340,2024-01-01,0,NaN,1825,1825,0,730
1,UNKNOWN,478095340,2024-02-01,0,0.00,1825,1825,0,730
2,UNKNOWN,478095340,2025-01-01,0,0.00,1825,1825,0,730
3,UNKNOWN,478095340,2025-02-01,0,0.00,1825,1825,0,730
4,UNKNOWN,478095340,2025-03-01,0,0.00,1825,1825,0,730
